In [1]:
import sys
# import os
# module_path = os.path.abspath(os.path.join('/home/owen/Lighthouse/w5/Midterm-Project-Lighthouselabs-/scripts'))
# if module_path not in sys.path:
#     sys.path.append(module_path)
sys.path.append("..")  # <-  This should point to the root directory of the project relative to this file
# import scripts
from scripts_test import database
# from scripts import preprocessing







In [2]:
example_query =  """
                SELECT * FROM flights
                    LIMIT 10;
                """

example = database.query(example_query)

In [3]:
example

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name
0,2018-02-26,AA,AA,AA,767,AA,N154UW,767,14107,PHX,...,1773.0,NaN,NaN,NaN,NaN,NaN,None,None,None,None
1,2018-02-26,AA,AA,AA,768,AA,N922US,768,13204,MCO,...,861.0,NaN,NaN,NaN,NaN,NaN,None,None,None,None
2,2018-02-26,AA,AA,AA,768,AA,N922US,768,14100,PHL,...,861.0,NaN,NaN,NaN,NaN,NaN,None,None,None,None
3,2018-02-26,AA,AA,AA,771,AA,N973UY,771,14107,PHX,...,2133.0,NaN,NaN,NaN,NaN,NaN,None,None,None,None
4,2018-02-26,AA,AA,AA,772,AA,N965UW,772,11042,CLE,...,363.0,NaN,NaN,NaN,NaN,NaN,None,None,None,None
5,2018-02-26,AA,AA,AA,772,AA,N965UW,772,14100,PHL,...,363.0,NaN,NaN,NaN,NaN,NaN,None,None,None,None
6,2018-02-26,AA,AA,AA,773,AA,N959UW,773,14100,PHL,...,363.0,0.0,0.0,0.0,0.0,26.0,None,None,None,None
7,2018-02-26,AA,AA,AA,774,AA,N710UW,774,13198,MCI,...,1038.0,NaN,NaN,NaN,NaN,NaN,None,None,None,None
8,2018-02-26,AA,AA,AA,774,AA,N710UW,774,14100,PHL,...,1038.0,NaN,NaN,NaN,NaN,NaN,None,None,None,None
9,2018-02-26,AA,AA,AA,775,AA,N809AW,775,12892,LAX,...,954.0,NaN,NaN,NaN,NaN,NaN,None,None,None,None


In [3]:
flights = preprocessing.get_training_flight_data()

In [5]:
flights

,fl_date,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,crs_arr_time,crs_elapsed_time,dup,flights,distance
0,2019-01-01,OO,N724SK,3200,13930,15370,1648,1855,127.0,N,1.0,585.0
1,2019-01-03,MQ,N820AE,3607,15074,11298,1152,1304,72.0,N,1.0,225.0
2,2019-01-03,MQ,N248NN,3680,14027,11278,1450,1717,147.0,N,1.0,857.0
3,2019-01-01,OH,N575NN,5606,13577,11057,2157,2311,74.0,N,1.0,157.0
4,2019-01-01,OH,N254PS,5607,10529,11278,1151,1325,94.0,N,1.0,313.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1239863,2018-01-30,G4,301NV,132,10466,14314,640,817,97.0,N,1.0,477.0
1239864,2018-01-30,G4,301NV,133,14314,10466,857,1031,94.0,N,1.0,477.0
1239865,2018-01-30,G4,304NV,514,12889,11638,700,811,71.0,N,1.0,259.0
1239866,2018-01-30,G4,304NV,515,11638,12889,851,958,67.0,N,1.0,259.0
